# BIOS 823

- The time allocated is 2 hours
- This is a **closed book** examination
    - Close ALL applications on your laptop
    - Start an empty browser with a SINGLE Tab in FULL SCREEN MODE
    - You should only have this SINGLE notebook page open in your browser, with NO OTHER TABS or WINDOWS
- You are not allowed any reference material except for the following:
    - Cheat sheet (1 letter-sized paper, both sides)
    - Built-in help accessible either by `?foo`, `foo?` or `help(foo)`
- ALL necessary imports of Python modules have been done for you. 
- **You should not import any additional modules - this includes standard library packages**.

Note that answers will be graded on **correctness**, **efficiency** and **readability**.

<font color=blue>By taking this exam, you acknowledge that you have read the instructions and agree to abide by the Duke Honor Code.</font>

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**1**. (10 points)

Warm up exercise.

Find the 5 most common words and their counts in `data/moby.txt`, after removing punctuation, setting to lowercase and splitting by blank space.

In [2]:
import string

In [17]:
string?

In [25]:
with open("data/moby.txt","r") as f:
    lines = f.readlines()
wordlist = [x.strip(string.punctuation).lower() for x in lines[1].split()]
wordset = set(wordlist)
wordcounts = [wordlist.count(word) for word in wordset]

In [37]:
sorted([(count,word) for count,word in zip(wordcounts, wordset)],reverse=True)[:5]

[(10, 'the'), (9, 'i'), (7, 'and'), (5, 'to'), (5, 'me')]

**2**. (10 points)

- Assemble the data from `features`, `subjects`, `X`, and `y` into a single `pandas.DataFrame (DF)` called `har`.  You should end up with a DF that is 7352 by 562 with `activity` as the first column. Rows and columns should be appropriately labeled.
    - `X` is a matrix where each row is a feature matrix
    - The columns of X are given in `features`
    - Each row of X is a subject given in `subjects`
    - `y` is a code for the type of activity performed by the subject (name the column in the DataFrame `actvitity`)
- Name the index `subject`
- Display a sample of 5 rows chosen at random without replacement and the first 5 columns.

In [39]:
activities = np.loadtxt('data/HAR/activity_labels.txt', dtype='str')
features = np.loadtxt('data/HAR/features.txt', dtype='str')[:, 1]
subjects = np.loadtxt('data/HAR/train/subject_train.txt', dtype='int')
#X = np.loadtxt('data/HAR/train/X_train.txt')
y = np.loadtxt('data/HAR/train/y_train.txt', dtype='int')

In [46]:
print(activities.shape,features.shape,subjects.shape,y.shape)

(6, 2) (561,) (7352,) (7352,)


In [44]:
pd.DataFrame(data = {"activities": activities, "features":features, "subjects":subjects, "y":y})

ValueError: arrays must all be same length

**3**. (10 points)

Using the DF from Question 1, find the average feature value for each subject for all features that have the string `entropy` in it but does NOT end in X, Y or Z. Use method chaining to perform this operation and show a random sample of 5 rows without replacement as a single expression.

**4**. (10 points)

Write an SQL query against the `har` table to count the number of distinct subjects and the total number of rows for each activity, ordering the results by number of rows for each activity in decreasing order. A simple example of how to run an SQL query using `pandas` is provided.

In [47]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///data/har.db', echo=False)

In [48]:
query = '''
SELECT subject, activity 
FROM har 
LIMIT 5
'''
pd.read_sql(query, con=engine)

,subject,activity
0,1,5.0
1,1,5.0
2,1,5.0
3,1,5.0
4,1,5.0


In [64]:

query = """
SELECT activity, COUNT(DISTINCT(subject)), COUNT(*) as rows
FROM har
GROUP BY activity
ORDER BY rows DESC
"""

pd.read_sql(query, con=engine)



,activity,COUNT(DISTINCT(subject)),rows
0,6.0,21,1407
1,5.0,21,1374
2,4.0,21,1286
3,1.0,21,1226
4,2.0,21,1073
5,3.0,21,986


**5**. (25 points)

- Create a new DF `df` from the `har` DF with all features that include the string `Acc-mean`
- Scale the feature columns so that all features have mean 0 and standard deviation 1
- Use SVD to find the first two principal components
- Plot the first two principal components as a scatter plot colored by the `activity` type of each feature vector
- Plot the 2D t-SNE plot colored in the same way (t-SNE dimension reduction may take 1-2 minutes)

Do not import any other packages apart from the cell below.

In [ ]:
from scipy.linalg import svd
from sklearn.manifold import TSNE

In [ ]:
activities

In [ ]:
X_test_data = np.loadtxt('data/HAR/test/X_test.txt')
y_test_data = np.loadtxt('data/HAR/test/y_test.txt', dtype='int')
subjects_test = np.loadtxt('data/HAR/test/subject_test.txt', dtype='int')

**6**. (25 points)

You are given training and test data and labels using a subset of the HAR data set. Your job is to use these features to classify rows into WALKING UPSTAIRS (code = 2) or WALKING DOWNSTAIRS (code = 3). 

- Scale the data to have mean zero and unit standard deviation using `StandardScaler`, taking care to apply the same scaling parameters for the training and test data sets
- Use the LaeblEncoder to transform the codes 2 and 3 to 0 and 1 in `y_train` and `y_test` 
- Perform ridge regression to classify data as WALKING UPSTAIRS or WALKING DOWNSTAIRS
    - Train the model with an Cs value chosen from one of (0.01, 0.1, 1, 10, 100) by 5-fold cross-validation using the training data
    - Plot the ROC curve (TPR versus FPR) evaluated on the test data

The necessary classes from `sklearn` are imported for you. Do not use any other `sklearn` classes

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_curve

In [ ]:
X_train = np.load('data/X_train.npy')
X_test = np.load('data/X_test.npy')
y_train = np.load('data/y_train.npy')
y_test = np.load('data/y_test.npy')